<a href="https://colab.research.google.com/github/msmsm104/MachineLearning_Project/blob/main/Training_Model/Modeling_4%E1%84%8E%E1%85%A1_0614.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML_Project(Modeling_4)

     => 데이터 전처리 방식에 문제가 있다는 것을 인지.
     => 가장 기본적으로 matchId, groupId를 기준으로 column별 평균, 평균의 rank 값으로만 전처리된 데이터로 모델링 진행.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
base_path = '/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/20220607/ML-project/data/Interim/'

## data optimize
#### train data set의 row가 400만개 이상이므로 Kaggle에서 제공하는 코드를 통해 data의 memory usage를 줄인 다음 modeling 작업을 진행한다. (그렇지 않는다면 중간에 kernel이 끊기는 문제가 발생할 수 있다.)
#### 좀 더 코드를 간소화하기 위해 train_test_split과 modeling과정을 함수로 정의하고 적용.

In [4]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## Load data

In [5]:
train_4 = reduce_mem_usage(pd.read_csv(base_path + 'featured_train_4.csv')) ## 4차 전처리 데이터


Memory usage of dataframe is 726.75 MB
Memory usage after optimization is: 181.69 MB
Decreased by 75.0%


## train_test_split

In [6]:
from sklearn.model_selection import train_test_split

def check_error(model_, data_):

    X = data_.drop(columns = 'winPlacePerc')
    y = data_['winPlacePerc']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

    model = model_
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return mean_absolute_error(y_test, pred)

## Modeling

    - LinearRegression
    - Lasso
    - Ridge
    - SVR(x)
    - KNeighborsRegressor(x)
    - DecisionTreeRegressor(x)
    - LGBMRegressor
    - RandomForestRegressor
    - XGBRegressor
    - CatBoostRegressor(x)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor

from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
# from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error

## Linear Reg, Lasso, Ridge, LGBMRegressor, RandomFor, XGBReg, CatBoost

### Linear Regression

In [8]:
result_1 = check_error(LinearRegression(), train_4)


print(f"MAE of LinearRegression: {result_1:.4f}")

# print(f"Difference : {result_2 - result_1:.4f}")

MAE of LinearRegression: 0.0581


### Lasso

In [9]:
result_1 = check_error(Lasso(), train_4)

print(f"MAE of Lasso_1: {result_1:.4f}")


# print(f"Difference : {result_2 - result_1:.4f}")

MAE of Lasso_1: 0.1148


### Ridge

In [10]:
result_1 = check_error(Ridge(), train_4)

print(f"MAE of Ridge_1: {result_1:.4f}")

# print(f"Difference : {result_2 - result_1:.4f}")

MAE of Ridge_1: 0.0581


### lightGBM

In [11]:
result_1 = check_error(LGBMRegressor(), train_4)


print(f"MAE of LGBMRegressor_1: {result_1:.4f}")

# print(f"Difference : {result_2 - result_1:.4f}")

MAE of LGBMRegressor_1: 0.0467


## 4차 전처리 데이터 분석.

    => 이상치, 컬럼 제거를 제외하고 matchId, groupId를 기준으로 컬럼값의 평균, 평균랭크값을 기반으로 모델의 학습을 진행.
    
    
    => 가장 성능이 괜찮은 모델, 분석하고 싶은 모델을 기반으로 hyper-parameter tuning을 통한 성능 개선 작업 시행 예정.